In [ ]:
import pandas as pd

In [ ]:
input_path = '../tmp-data/ss_range_bars.1'
known_false_signals = False

In [ ]:
df = pd.read_csv(f'{input_path}.csv', parse_dates=['timestamp'], index_col='timestamp')
df

In [ ]:
from backtesting import Strategy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

class RangeBarStrategyPreCalc(Strategy):

    per_trade_risk = 0.1
    
    def init(self):
       self.false_buys = 0
       self.false_sells = 0

    def next(self):
        current_close = self.data.Close[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        pre_calc_signal = self.data.signal[-1]
        pre_calc_false_signal = self.data.false_signal[-1]

        known_false_pass = True
        if known_false_signals:
            known_false_pass = pre_calc_false_signal != 1

        if pre_calc_signal == 1 and known_false_pass:
            if pre_calc_signal == 1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal buy')
                    self.false_buys += 1
            else:
                print(f'ERROR: buying, against pre_calc_signal')    
            self.buy(size=self.per_trade_risk, sl=current_close - range_size, tp=current_close + potential_profit)
        elif pre_calc_signal == -1 and known_false_pass:
            if pre_calc_signal == -1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal sell')
                    self.false_sells += 1  
            else:
                print(f'ERROR: selling, against pre_calc_signal')
            self.sell(size=self.per_trade_risk, sl=current_close + range_size , tp=current_close - potential_profit)

## known false signals

| Return % | risk po | start | end | false b | false s | tot trades | max dd | win rate |
| -------- | ------- | ----- | --- | ------- | ------- | ---------- | ------ | -------- |
| 12,537.59 | 0.1 | 2022-03-21 23:10:00 | 2022-07-02 21:02:0 | 0 | 0 | 4796.0 | -18.252684 | 49.582986 |

## unknown false signals

| Return % | risk po | start | end | false b | false s | tot trades | max dd | win rate |
| -------- | ------- | ----- | --- | ------- | ------- | ---------- | ------ | -------- |
| 3,590.15 | 0.1 | 2022-03-21 23:10:00 | 2022-07-02 21:02:0 | 451 | 530 | 5773 | -22.111453 | 42.0925 |

In [ ]:
# df_sample = df
# start = df_sample.iloc[0]['timestamp']
# end = df_sample.iloc[-1]['timestamp']
# print(f'start: {start}, end: {end}')

## Periods of negative return

| Return % | start | start idx | end | end idx |
| -------- | ----- | --------- | --- | ------- | 
| -34.11704 | 2022-03-27 12:47:00 | 1500 | 2022-04-12 01:22:00 | 6500 |
| -59.653264 |2022-03-26 07:52:00| 1418 | 2022-04-27 01:25:00 | 10500 |

## Periods of positive return

| Return % | start | start idx | end | end idx |
| -------- | ----- | --------- | --- | ------- | 
| 649.93 | 2022-04-27 01:26:00 | 10500 | 2022-05-31 18:09:00 | 20500 |
| 1231.28 | 2022-05-31 18:09:00 | 20500 | 2022-07-05 12:46:00 | 30500 |
| 1875.42 | 2022-07-05 12:47:00 | 30500 | 2022-08-10 16:41:00 | 40500 |


check assumption that strat1 doesn't work with start index > 0 because precalculated indications are off, by recalc on section

The full chart shows 80% max draw down, need to discover why the strategy fails in these periods and how to opt out of trading. The number is false signals seems to be ~ consistent % of total trades

In [ ]:
from backtesting import Backtest
bt = Backtest(df, RangeBarStrategyPreCalc, cash=100000, margin=1/10, commission=(0.03 / 100), exclusive_orders=False)
stats = bt.run()
false_buys = stats._strategy.false_buys
false_sells = stats._strategy.false_sells
print(f'false buys: {false_buys}, false sells: {false_sells}')
stats


Sharpe Ratio                          0.01437
Sortino Ratio                    85554.785001
Calmar Ratio                    143047.418449

